<a href="https://colab.research.google.com/github/aastha12/dphi-machine-learning-bootcamp/blob/main/Heart_Disease_Binary_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 102kB 2.7MB/s 


In [ ]:
from skopt import BayesSearchCV

## Load the data
Display the first 5 rows of the data after loading.

In [ ]:
train_df  = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/Heart_Disease/Training_set_heart.csv" )

train_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,48,1,2,124,255,1,1,175,0,0.0,2,2,2,1
1,68,0,2,120,211,0,0,115,0,1.5,1,0,2,1
2,46,1,0,120,249,0,0,144,0,0.8,2,0,3,0
3,60,1,0,130,253,0,1,144,1,1.4,2,1,3,0
4,43,1,0,115,303,0,1,181,0,1.2,1,0,2,1


In [ ]:
test_df = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/Heart_Disease/Testing_set_heart.csv')

## Perform Basic Exploratory Data Analysis

In [ ]:
"""
Using sweetviz which is an automatic EDA library to analyse the dataset first.
I will also look at correlation, histogram, missing values, etc. later on.
"""
!pip install sweetviz

In [ ]:
# importing sweetviz
import sweetviz as sv

#analyzing the dataset
advert_report = sv.analyze(train_df)
#display the report
advert_report.show_html('Heart_Disease.html')


Report Heart_Disease.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
train_df.describe(include='all')

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000,212.000000
mean,54.561321,0.688679,0.915094,132.127358,247.830189,0.165094,0.537736,148.995283,0.330189,1.008491,1.448113,0.783019,2.320755,0.542453
std,9.493376,0.464130,1.008193,17.440700,53.199877,0.372144,0.527437,23.332645,0.471394,1.141681,0.601850,1.066551,0.646495,0.499374
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.000000,0.000000,0.000000,120.000000,211.750000,0.000000,0.000000,132.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,57.000000,1.000000,1.000000,130.000000,240.500000,0.000000,1.000000,151.500000,0.000000,0.650000,2.000000,0.000000,2.000000,1.000000
75%,61.250000,1.000000,2.000000,140.000000,277.000000,0.000000,1.000000,165.250000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,192.000000,564.000000,1.000000,2.000000,202.000000,1.000000,5.600000,2.000000,4.000000,3.000000,1.000000


In [ ]:
train_df.isnull().sum() #no missing values

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [ ]:
train_df.skew() #chol, fbs, oldpeak and ca columns have skewness. we will confirm with histograms

age        -0.268034
sex        -0.820789
cp          0.535785
trestbps    0.571733
chol        1.302705
fbs         1.817011
restecg     0.141633
thalach    -0.474609
exang       0.727326
oldpeak     1.249342
slope      -0.588684
ca          1.224102
thal       -0.634908
target     -0.171644
dtype: float64

In [ ]:
train_df.dtypes #checking data type of columns. All are numerical columns

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [ ]:
train_df.shape #training set has 212 rows and 14 features

(212, 14)

In [ ]:
train_df['target'].value_counts(normalize=True) #somewhat balanced dataset

1    0.542453
0    0.457547
Name: target, dtype: float64

In [ ]:
"""
Positive correlation of target with - chest pain and max heart rate achieved
Negative correlation of target with - exercise induced angina and old peak
"""
train_df.corr()['target'].sort_values() 

exang      -0.482622
oldpeak    -0.398818
ca         -0.365253
thal       -0.291939
sex        -0.249428
age        -0.201492
trestbps   -0.114081
chol       -0.086962
fbs        -0.050644
restecg     0.200817
slope       0.370050
thalach     0.417547
cp          0.477865
target      1.000000
Name: target, dtype: float64

In [ ]:
index = 0

fig = make_subplots(rows=7,cols=2,subplot_titles=(train_df.columns))

for i in range(1,8):
  for j in range(1,3):
    plot_df = train_df[train_df.columns[index]]
    trace = go.Histogram(x=plot_df)
    fig.append_trace(trace,i,j)
    index+=1

fig.update_layout(height=900,width=1250,title_text='Distribution of Numerical Attributes')
fig.show()

In [ ]:
index = 0

fig = make_subplots(rows=7,cols=2,subplot_titles=(train_df.columns))

for i in range(1,8):
  for j in range(1,3):
    plot_df = train_df[train_df.columns[index]]
    trace = go.Box(x=plot_df)
    fig.append_trace(trace,i,j)
    index+=1

fig.update_layout(height=900,width=1250,title_text='Boxplot of Numerical Attributes')
fig.show()

From the histogram and boxplots, we can see that some of the features are skewed and some contain outliers like trestbps, oldpeak, ca, and thalach. We can also see that all the features are at a different scale which means we need to perform scaling before modeling

## Separate the Input and Target Features of the data

In [ ]:
X = train_df.copy()
X.drop(['target'],axis=1,inplace=True)
y=train_df['target']

## Split the data into Train and Test Sets
The train to test ratio should be 80:20 and the random_state should be 0.


In [ ]:
X_train,X_valid,y_train,y_valid=train_test_split(X,y,random_state=0,test_size=0.2)
print("Shape of training features:",X_train.shape)
print("Shape of training labels:",y_train.shape)
print("Shape of testing features:",X_valid.shape)
print("Shape of testing labels:",y_valid.shape)

Shape of training features: (169, 13)
Shape of training labels: (169,)
Shape of testing features: (43, 13)
Shape of testing labels: (43,)


## Transform data

In [ ]:
pt= PowerTransformer(standardize=False) #by default standardize is True which means it will change the mean to 0 and variance to 1

#pt = QuantileTransformer(output_distribution='normal',random_state=0)
X_train_transformed=pt.fit_transform(X_train)
X_valid_transformed=pt.transform(X_valid)

In [ ]:
X_test_transformed=pt.transform(test_df)

## Scaling

In [ ]:
rs=RobustScaler()

#rs=MinMaxScaler()

X_train_scaled=rs.fit_transform(X_train_transformed)
X_valid_scaled=rs.transform(X_valid_transformed)

In [ ]:
X_test_scaled=rs.transform(X_test_transformed)

## Distribution of transformed data

In [ ]:
train_scaled_df = pd.DataFrame(X_train_scaled,columns=X.columns)
test_scaled_df = pd.DataFrame(X_test_scaled,columns =X.columns)
valid_scaled_df = pd.DataFrame(X_valid_scaled,columns =X.columns)

index = 0

fig = make_subplots(rows=7,cols=2,subplot_titles=(X.columns))

for i in range(1,7):
  for j in range(1,3):
    plot_df = train_scaled_df[train_scaled_df.columns[index]]
    trace = go.Histogram(x=plot_df)
    fig.append_trace(trace,i,j)
    index+=1

fig.update_layout(height=900,width=1250,title_text='Distribution of Training Set')
fig.show()

## Plotting data 

In [ ]:
from sklearn.decomposition import PCA

pca=PCA(n_components=2)

plot_train= pca.fit_transform(train_scaled_df)

fig=go.Figure()
fig.add_trace(go.Scatter(x=plot_train[:,0],y=plot_train[:,1],mode='markers',marker_color=y))
fig.show()

## Feature Selection using SelectFromModel

In [ ]:
train_scaled_df.head(2)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0.127744,0.0,-0.693204,0.881672,0.407012,-0.0,-1.0,-1.098745,1.0,0.120863,0.0,-0.0,1.0
1,-0.375246,-1.0,-0.693204,-1.145740,0.172727,-0.0,-1.0,0.156289,-0.0,-0.587693,0.0,-0.0,0.0


In [ ]:
"""
Using Logistic Regression and SelectFromModel to select the important features
"""

selector=SelectFromModel(estimator=LogisticRegression(random_state=0))
selector.fit(train_scaled_df,y_train)

selected_cols=[]

for selected,col in zip(selector.get_support(),train_scaled_df.columns):
  print(col," has been selected?",selected)
  if selected==True:
    selected_cols.append(col)

age  has been selected? False
sex  has been selected? False
cp  has been selected? True
trestbps  has been selected? False
chol  has been selected? False
fbs  has been selected? False
restecg  has been selected? True
thalach  has been selected? True
exang  has been selected? True
oldpeak  has been selected? False
slope  has been selected? False
ca  has been selected? True
thal  has been selected? True


In [ ]:
selected_cols

['cp', 'restecg', 'thalach', 'exang', 'ca', 'thal']

## Polynomial Features

In [ ]:
train_scaled_df.head(2)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0.127744,0.0,-0.693204,0.881672,0.407012,-0.0,-1.0,-1.098745,1.0,0.120863,0.0,-0.0,1.0
1,-0.375246,-1.0,-0.693204,-1.145740,0.172727,-0.0,-1.0,0.156289,-0.0,-0.587693,0.0,-0.0,0.0


In [ ]:
poly=PolynomialFeatures(degree=2,include_bias=False)

"""
taking degree 2 of the selected columns. Chose "2" as it gave higher F1 score
"""

poly_transformed_train= poly.fit_transform(train_scaled_df[['cp', 'restecg', 'thalach', 'exang', 'ca', 'thal']])
poly_transformed_valid= poly.transform(valid_scaled_df[['cp', 'restecg', 'thalach', 'exang', 'ca', 'thal']])
poly_transformed_test= poly.transform(test_scaled_df[['cp', 'restecg', 'thalach', 'exang', 'ca', 'thal']])


## Modeling

In [ ]:
LR = LogisticRegression(random_state=0)
SVM = SVC(random_state=0)
XGB=XGBClassifier(random_state=0)

lr_score=cross_val_score(LR,poly_transformed_train,y_train,scoring='f1',cv=5)
svm_score=cross_val_score(SVM,poly_transformed_train,y_train,scoring='f1',cv=5)
xgb_score=cross_val_score(XGB,poly_transformed_train,y_train,scoring='f1',cv=5)

In [ ]:
print("Avg F1 score for LR:",lr_score.mean())
print("Avg F1 score for SVM:",svm_score.mean())
print("Avg F1 score for XGBoost:",xgb_score.mean())

Avg F1 score for LR: 0.8663069307545636
Avg F1 score for SVM: 0.8689915966386554
Avg F1 score for XGBoost: 0.8647374847374847


We will choose Logistic Regression as it has the highest F1 score.

## Multicolinearity for Logistic Regression

In [ ]:
"""
from statsmodels.stats.outliers_influence import variance_inflation_factor

#converting to df as vif requires(value, column index)-https://www.statsmodels.org/stable/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html
X_train_scaled.shape
vif=[variance_inflation_factor(X_train_scaled,i) for i in range(X_train_scaled.shape[1])]
"""

'\nfrom statsmodels.stats.outliers_influence import variance_inflation_factor\n\n#converting to df as vif requires(value, column index)-https://www.statsmodels.org/stable/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html\nX_train_scaled.shape\nvif=[variance_inflation_factor(X_train_scaled,i) for i in range(X_train_scaled.shape[1])]\n'

In [ ]:
#vif

No multicolinearity present

## Build a Logistic Regression Model on train set

In [ ]:
LR=LogisticRegression(random_state=0)

LR.fit(poly_transformed_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Evaluate the model using F1 Score

In [ ]:
predictions=LR.predict(poly_transformed_valid)

f1_score(y_valid,predictions)

0.8571428571428572

In [ ]:
print(classification_report(y_valid,predictions))

              precision    recall  f1-score   support

           0       0.79      0.83      0.81        18
           1       0.88      0.84      0.86        25

    accuracy                           0.84        43
   macro avg       0.83      0.84      0.83        43
weighted avg       0.84      0.84      0.84        43



In [ ]:
X_valid_scaled.shape,X_train_scaled.shape

((43, 13), (169, 13))

## Error Analysis

In [ ]:
y_valid.values

array([1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1])

In [ ]:
X_valid_copy = X_valid_scaled
y_valid_copy = y_valid.values

result=pd.DataFrame(X_valid_copy,columns=X.columns)

result['prediction']=predictions
result['actual']=y_valid_copy

result['Error']= result.apply(lambda x: 1 if x.prediction!=x.actual else 0,axis=1)
result.sort_values(by='Error',ascending=False)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,prediction,actual,Error
21,0.090015,0.0,0.000000,0.920815,-0.232002,0.000000,-1.000000,0.405652,0.0,-0.891807,0.00000,1.000000,0.000000,1,0,1
6,0.288070,0.0,0.810860,1.613391,0.435600,0.000000,-1.000000,-0.205442,0.0,0.433682,-1.78713,0.000000,1.000000,0,1,1
31,0.791146,0.0,-0.881852,-1.052389,0.083707,0.000000,-1.000000,0.127122,0.0,0.000000,0.00000,1.093763,-0.256351,1,0,1
16,-0.298398,0.0,0.810860,-0.294968,-0.510498,0.000000,-1.000000,-0.768363,1.0,0.095841,0.00000,1.000000,0.000000,0,1,1
15,0.288070,0.0,0.810860,1.422309,0.637817,0.000000,-1.000000,0.160814,0.0,-0.042371,-1.00000,0.000000,1.000000,1,0,1
11,-0.175104,0.0,-0.881852,0.564584,-0.321524,0.000000,-1.000000,-1.140364,1.0,-0.891807,0.00000,0.000000,1.000000,0,1,1
14,0.752554,0.0,-0.881852,-0.189391,0.297250,0.000000,0.000000,-1.266552,1.0,-0.042371,-1.00000,1.000000,1.000000,0,1,1
32,-0.443165,0.0,-0.881852,-0.844509,-0.654012,0.000000,0.000000,-0.292283,0.0,-0.052906,0.00000,0.000000,0.000000,1,1,0
26,0.198948,-1.0,-0.881852,1.422309,-0.327645,10.398675,-1.000000,-0.188432,1.0,0.232376,-1.00000,1.093763,-0.256351,0,0,0
27,0.791146,-1.0,-0.881852,0.754596,-0.327645,0.000000,-1.000000,-1.029108,0.0,0.043935,-1.00000,1.196143,1.000000,0,0,0


In [ ]:
result[result['Error']==1]

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,prediction,actual,Error
6,0.288070,0.0,0.810860,1.613391,0.435600,0.0,-1.0,-0.205442,0.0,0.433682,-1.78713,0.000000,1.000000,0,1,1
11,-0.175104,0.0,-0.881852,0.564584,-0.321524,0.0,-1.0,-1.140364,1.0,-0.891807,0.00000,0.000000,1.000000,0,1,1
14,0.752554,0.0,-0.881852,-0.189391,0.297250,0.0,0.0,-1.266552,1.0,-0.042371,-1.00000,1.000000,1.000000,0,1,1
15,0.288070,0.0,0.810860,1.422309,0.637817,0.0,-1.0,0.160814,0.0,-0.042371,-1.00000,0.000000,1.000000,1,0,1
16,-0.298398,0.0,0.810860,-0.294968,-0.510498,0.0,-1.0,-0.768363,1.0,0.095841,0.00000,1.000000,0.000000,0,1,1
21,0.090015,0.0,0.000000,0.920815,-0.232002,0.0,-1.0,0.405652,0.0,-0.891807,0.00000,1.000000,0.000000,1,0,1
31,0.791146,0.0,-0.881852,-1.052389,0.083707,0.0,-1.0,0.127122,0.0,0.000000,0.00000,1.093763,-0.256351,1,0,1


## Feature Selection for LR

In [ ]:
selector=SelectFromModel(estimator=LogisticRegression(random_state=0))
selector.fit(poly_transformed_train,y_train)

selected_cols=[]

for selected,col in zip(selector.get_support(),X.columns):
  print(col," has been selected?",selected)
  if selected==True:
    selected_cols.append(col)

age  has been selected? True
sex  has been selected? False
cp  has been selected? True
trestbps  has been selected? True
chol  has been selected? True
fbs  has been selected? True
restecg  has been selected? True
thalach  has been selected? False
exang  has been selected? False
oldpeak  has been selected? True
slope  has been selected? True
ca  has been selected? False
thal  has been selected? False


In [ ]:
selector.estimator_.coef_

array([[ 0.81831523,  0.12044949,  0.76005052, -0.70434683, -0.62474608,
        -0.5332913 , -0.5197273 , -0.22098445,  0.03434859,  0.79082212,
         0.62932361,  0.06674397, -0.1151142 ,  0.09939892, -0.57775078,
         1.20466048,  0.85732855, -0.52462971,  0.41142081, -0.03221716,
         0.0522707 , -0.70434683, -0.1889006 , -0.04879083, -0.04169752,
         0.39970932, -1.55318236]])

In [ ]:
poly_transformed_valid.shape,poly_transformed_train.shape

((43, 27), (169, 27))

In [ ]:
X_train_selected=selector.transform(poly_transformed_train)

In [ ]:
X_train_selected.shape

(169, 14)

In [ ]:
poly_transformed_valid.shape,poly_transformed_train.shape

((43, 27), (169, 27))

In [ ]:
LR=LogisticRegression(random_state=0)
LR.fit(X_train_selected,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
X_valid_selected=selector.transform(poly_transformed_valid)
predictions=LR.predict(X_valid_selected)

f1_score(y_valid,predictions)

0.8461538461538461

In [ ]:
#replicating for test set

X_test_selected=selector.transform(poly_transformed_test)

In [ ]:
print(classification_report(y_valid,predictions))

              precision    recall  f1-score   support

           0       0.81      0.72      0.76        18
           1       0.81      0.88      0.85        25

    accuracy                           0.81        43
   macro avg       0.81      0.80      0.81        43
weighted avg       0.81      0.81      0.81        43



In [ ]:
"""
X_valid_copy = X_valid_selected
y_valid_copy = y_valid.values

result=pd.DataFrame(X_valid_copy,columns=selected_cols)

result['prediction']=predictions
result['actual']=y_valid_copy

result['Error']= result.apply(lambda x: 1 if x.prediction!=x.actual else 0,axis=1)
result.sort_values(by='Error',ascending=False)
"""

"\nX_valid_copy = X_valid_selected\ny_valid_copy = y_valid.values\n\nresult=pd.DataFrame(X_valid_copy,columns=selected_cols)\n\nresult['prediction']=predictions\nresult['actual']=y_valid_copy\n\nresult['Error']= result.apply(lambda x: 1 if x.prediction!=x.actual else 0,axis=1)\nresult.sort_values(by='Error',ascending=False)\n"

In [ ]:
#result[result['Error']==1]

## Hyperparameter Tuning for Logistic Regression

In [ ]:
params = {'C': (1e-2, 100.0, 'log-uniform')}

In [ ]:
LR=LogisticRegression(random_state=0,max_iter= 200)

#using the polynomial transformed training set as it gave better results
LR.fit(poly_transformed_train,y_train)

tuned_LR=BayesSearchCV(LR,params,cv=4,random_state=0,scoring='f1')

tuned_LR.fit(poly_transformed_train,y_train)

BayesSearchCV(cv=4, estimator=LogisticRegression(max_iter=200, random_state=0),
              random_state=0, scoring='f1',
              search_spaces={'C': (0.01, 100.0, 'log-uniform')})

In [ ]:
tuned_LR.best_score_

0.8872229788760068

In [ ]:
tuned_LR.best_params_

OrderedDict([('C', 0.583837242413568)])

In [ ]:
predictions=tuned_LR.predict(poly_transformed_valid)

f1_score(y_valid,predictions)

0.8627450980392156

## Hyperparameter tuning for SVM

In [ ]:
SVM = SVC(random_state=0)

SVM.fit(poly_transformed_train,y_train)

"""
selector=SelectFromModel(SVM)
selector.fit(X_train_scaled,y_train)

selected_cols=[]

for selected,col in zip(selector.get_support(),X.columns):
  print(col," has been selected?",selected)
  if selected==True:
    selected_cols.append(col)

X_train_selected=selector.transform(X_train_scaled)    
X_valid_selected=selector.transform(X_valid_scaled)
X_test_selected=selector.transform(X_test_scaled)

SVM=SVC(random_state=0)
SVM.fit(X_train_selected,y_train)
predictions=SVM.predict(X_valid_selected)

print("F1 score for SVM After feature selection and before tuning:",f1_score(y_valid,predictions))
"""

#The strength of the regularization is inversely proportional to C.
#reducing C to increase regularization and avoid overfitting

tuned_parameters = [{'kernel': ['rbf','sigmoid','linear'],
                     'C':[0.10,1]
                    }
                   ]

tuned_SVM=BayesSearchCV(SVM,tuned_parameters,cv=4,random_state=0,scoring='f1')

tuned_SVM.fit(poly_transformed_train,y_train)
print("Best 4 fold F1 score after tuning for SVM:",tuned_SVM.best_score_)

predictions=tuned_SVM.predict(poly_transformed_valid)

print("F1 score after tuning",f1_score(y_valid,predictions))

Best 4 fold F1 score after tuning for SVM: 0.8811397810188104
F1 score after tuning 0.8333333333333333


In [ ]:
tuned_SVM.best_params_

OrderedDict([('C', 0.5758225678679059), ('kernel', 'linear')])

## Hyperparameter tuning for Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF=RandomForestClassifier(random_state=0 )

cross_val_score(RF,poly_transformed_train,y_train,cv=4,scoring='f1').mean()

0.8717171717171717

In [ ]:
# Defining your params and training rf on entire training set
search_spaces = {'max_depth': [3, 4, 5, 6, 7, 8,9,10,11,12],
                 'n_estimators':[100,300],
                 'min_samples_split' : [5, 10]}

tuned_RF=BayesSearchCV(RF,search_spaces,cv=4,random_state=0,scoring='f1')

tuned_RF.fit(poly_transformed_train,y_train)
print("Best 4 fold F1 score after tuning for Random Forest:",tuned_RF.best_score_)

Best 4 fold F1 score after tuning for Random Forest: 0.889468391140632


## Hyperparameter Tuning for Catboost

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

cat=CatBoostClassifier( )

cross_val_score(cat,poly_transformed_train,y_train,cv=4,scoring='f1').mean()

Learning rate set to 0.004254
0:	learn: 0.6900356	total: 49.2ms	remaining: 49.1s
1:	learn: 0.6869325	total: 50.4ms	remaining: 25.2s
2:	learn: 0.6840559	total: 51.6ms	remaining: 17.1s
3:	learn: 0.6814862	total: 52.8ms	remaining: 13.1s
4:	learn: 0.6783963	total: 53.9ms	remaining: 10.7s
5:	learn: 0.6758771	total: 55ms	remaining: 9.12s
6:	learn: 0.6728850	total: 56.1ms	remaining: 7.96s
7:	learn: 0.6708367	total: 56.8ms	remaining: 7.04s
8:	learn: 0.6685201	total: 57.9ms	remaining: 6.37s
9:	learn: 0.6658210	total: 59ms	remaining: 5.84s
10:	learn: 0.6624001	total: 60ms	remaining: 5.4s
11:	learn: 0.6593396	total: 61.2ms	remaining: 5.04s
12:	learn: 0.6561265	total: 62.3ms	remaining: 4.73s
13:	learn: 0.6541398	total: 63.4ms	remaining: 4.46s
14:	learn: 0.6505958	total: 64.5ms	remaining: 4.24s
15:	learn: 0.6477827	total: 65.6ms	remaining: 4.03s
16:	learn: 0.6453810	total: 66.7ms	remaining: 3.86s
17:	learn: 0.6422730	total: 67.8ms	remaining: 3.7s
18:	learn: 0.6389599	total: 68.9ms	remaining: 3.56s


0.8756724857268336

In [ ]:
# Defining your search space
from skopt.space import Real, Categorical, Integer

search_spaces = {'iterations': Integer(10, 1000),
                 'depth': Integer(1, 8),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 'random_strength': Real(1e-9, 10, 'log-uniform'),
                 'bagging_temperature': Real(0.0, 1.0),
                 'border_count': Integer(1, 255),
                 'l2_leaf_reg': Integer(2, 30),
                 'scale_pos_weight':Real(0.01, 1.0, 'uniform')}

tuned_cat=BayesSearchCV(cat,search_spaces,cv=4,random_state=0,scoring='f1')

tuned_cat.fit(poly_transformed_train,y_train)
print("Best 4 fold F1 score after tuning for Catboost:",tuned_cat.best_score_)

Streaming output truncated to the last 5000 lines.
270:	learn: 0.3226970	total: 217ms	remaining: 409ms
271:	learn: 0.3218023	total: 218ms	remaining: 408ms
272:	learn: 0.3215437	total: 219ms	remaining: 407ms
273:	learn: 0.3211288	total: 220ms	remaining: 406ms
274:	learn: 0.3208257	total: 220ms	remaining: 405ms
275:	learn: 0.3203496	total: 221ms	remaining: 404ms
276:	learn: 0.3200545	total: 222ms	remaining: 403ms
277:	learn: 0.3200272	total: 222ms	remaining: 402ms
278:	learn: 0.3194833	total: 223ms	remaining: 401ms
279:	learn: 0.3188509	total: 224ms	remaining: 400ms
280:	learn: 0.3184520	total: 224ms	remaining: 399ms
281:	learn: 0.3183453	total: 225ms	remaining: 398ms
282:	learn: 0.3180458	total: 226ms	remaining: 397ms
283:	learn: 0.3175699	total: 227ms	remaining: 397ms
284:	learn: 0.3172192	total: 227ms	remaining: 396ms
285:	learn: 0.3167330	total: 228ms	remaining: 395ms
286:	learn: 0.3162421	total: 229ms	remaining: 394ms
287:	learn: 0.3160971	total: 230ms	remaining: 393ms
288:	learn: 0

## Hyperparameter tuning for XGBoost

In [ ]:
xgb=XGBClassifier(random_state=0)

xgb.fit(poly_transformed_train,y_train)

  # Defining your params and training xgboost on entire training set
search_spaces = {'max_depth': [3, 4, 5, 6, 7, 8,9,10,11,12],
                 'min_child_weight': np.arange(0.0001, 0.5, 0.001),
                 #'gamma': np.arange(0.2,40.0,0.5),
                 'learning_rate': np.arange(0.0005,0.3,0.0005),
                 #'subsample': np.arange(0.01,1.0,0.01),
                 'n_estimators':[100,300]}

tuned_XGB=BayesSearchCV(xgb,search_spaces,cv=4,random_state=0,scoring='f1')

tuned_XGB.fit(poly_transformed_train,y_train)
print("Best 4 fold F1 score after tuning for XGBoost:",tuned_XGB.best_score_)

predictions=tuned_XGB.predict(poly_transformed_valid)

print("F1 score after tuning",f1_score(y_valid,predictions))

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning:

The objective has been evaluated at this point before.

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning:

The objective has been evaluated at this point before.



Best 4 fold F1 score after tuning for XGBoost: 0.8697051720783309
F1 score after tuning 0.7727272727272727


In [ ]:
tuned_XGB.best_params_

OrderedDict([('learning_rate', 0.0125),
             ('max_depth', 3),
             ('min_child_weight', 0.4841),
             ('n_estimators', 299)])

## Neural Networks

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

In [ ]:
poly_transformed_train.shape

(169, 27)

In [ ]:
classifier_ann = Sequential()
classifier_ann.add(Dense(100,activation='relu',kernel_initializer='he_uniform',input_dim=poly_transformed_train.shape[1]))
classifier_ann.add(Dropout(0.5,seed=123))
classifier_ann.add(BatchNormalization())
classifier_ann.add(Dense(100,activation='relu',kernel_initializer='he_uniform'))
classifier_ann.add(Dropout(0.5,seed=123))
classifier_ann.add(BatchNormalization())
classifier_ann.add(Dense(100,activation='relu',kernel_initializer='he_uniform'))
classifier_ann.add(Dropout(0.5,seed=123))
classifier_ann.add(BatchNormalization())
classifier_ann.add(Dense(1,activation='sigmoid',kernel_initializer='glorot_uniform'))
classifier_ann.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_89 (Dense)             (None, 100)               2800      
_________________________________________________________________
dropout_68 (Dropout)         (None, 100)               0         
_________________________________________________________________
batch_normalization_67 (Batc (None, 100)               400       
_________________________________________________________________
dense_90 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_69 (Dropout)         (None, 100)               0         
_________________________________________________________________
batch_normalization_68 (Batc (None, 100)               400       
_________________________________________________________________
dense_91 (Dense)             (None, 100)             

In [ ]:
classifier_ann.compile(optimizer='adam',metrics=keras.metrics.AUC(name='auc'),loss='binary_crossentropy')

#early stopping to avoid overfitting

es=EarlyStopping(monitor='val_auc',patience=25,mode='max')

In [ ]:
poly_transformed_valid.shape

(43, 27)

In [ ]:
model_history=classifier_ann.fit(poly_transformed_train,y_train,epochs=120,
                   validation_data=(poly_transformed_valid,y_valid),
                   callbacks=es)

Epoch 1/120
6/6 [==============================] - 2s 78ms/step - loss: 0.9528 - auc: 0.4699 - val_loss: 0.6750 - val_auc: 0.6978
Epoch 2/120
6/6 [==============================] - 0s 13ms/step - loss: 0.8102 - auc: 0.5676 - val_loss: 0.6624 - val_auc: 0.7722
Epoch 3/120
6/6 [==============================] - 0s 13ms/step - loss: 0.7190 - auc: 0.6492 - val_loss: 0.6589 - val_auc: 0.7911
Epoch 4/120
6/6 [==============================] - 0s 13ms/step - loss: 0.7388 - auc: 0.6168 - val_loss: 0.6584 - val_auc: 0.8400
Epoch 5/120
6/6 [==============================] - 0s 13ms/step - loss: 0.8201 - auc: 0.5794 - val_loss: 0.6645 - val_auc: 0.8511
Epoch 6/120
6/6 [==============================] - 0s 12ms/step - loss: 0.6784 - auc: 0.6860 - val_loss: 0.6610 - val_auc: 0.8489
Epoch 7/120
6/6 [==============================] - 0s 14ms/step - loss: 0.6797 - auc: 0.7046 - val_loss: 0.6562 - val_auc: 0.8522
Epoch 8/120
6/6 [==============================] - 0s 13ms/step - loss: 0.5532 - auc: 0.79

In [ ]:
valid_pred=classifier_ann.predict_classes(poly_transformed_valid)
f1_score(y_valid,valid_pred)

0.8076923076923077

In [ ]:
fig=go.Figure()

x=[i for i in range(1,47)]

fig.add_trace(go.Scatter(x=x,y=model_history.history['auc'],mode='lines',name='Training AUC'))
fig.add_trace(go.Scatter(x=x,y=model_history.history['val_auc'],mode='lines',name='Valid AUC'))
fig.add_trace(go.Scatter(x=x,y=model_history.history['loss'],mode='lines',name='Training loss'))
fig.add_trace(go.Scatter(x=x,y=model_history.history['val_loss'],mode='lines',name='Valid loss'))

fig.show()

In [ ]:
test_pred=classifier_ann.predict_classes(poly_transformed_test)
res= pd.DataFrame(test_pred)

res.index=test_df.index
res.columns= ['prediction']

res.to_csv('ANN.csv',index=False)

## Test Predictions

In [ ]:
"""
Getting predictions for training set for tuned LR and tuned SVM
we will pass these predictions along with y training label for the ANN
"""
train_pred_LR=tuned_LR.predict(poly_transformed_train)

train_pred_SVM=tuned_SVM.predict(poly_transformed_train)

#train_pred_RF=tuned_RF.predict(poly_transformed_train)

train_pred_df=pd.DataFrame(train_pred_LR,columns=["LR"])
train_pred_df['SVM']=train_pred_SVM
#train_pred_df['RF']=train_pred_RF


In [ ]:
xgb=XGBClassifier(random_state=0)

cross_val_score(xgb,train_pred_df,y_train,cv=4,scoring='f1').mean()

0.8670903564520586

In [ ]:

xgb.fit(train_pred_df,y_train)

# Defining your params and training catboost on entire training set
search_spaces = {'max_depth': [3, 4, 5],
                 'min_child_weight': np.arange(0.0001, 0.5, 0.001),
                 'gamma': np.arange(0.2,40.0,0.5),
                 'learning_rate': np.arange(0.0005,0.3,0.0005),
                 #'subsample': np.arange(0.01,1.0,0.01),
                 'n_estimators':[50,100]}

tuned_XGB=BayesSearchCV(xgb,search_spaces,cv=4,random_state=0,scoring='f1')

tuned_XGB.fit(train_pred_df,y_train)
print("Best 4 fold F1 score after tuning for XGBoost:",tuned_XGB.best_score_)


Best 4 fold F1 score after tuning for XGBoost: 0.8873087348146055


In [ ]:
tuned_XGB.best_params_

OrderedDict([('gamma', 38.199999999999996),
             ('learning_rate', 0.2265),
             ('max_depth', 4),
             ('min_child_weight', 0.0921),
             ('n_estimators', 62)])

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(tuned_XGB.best_estimator_, train_pred_df, y_train, cv=4,
                                                                      return_times=True,random_state=123,
                                                                     scoring='f1')
    
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)


# Plot learning curve
import plotly.graph_objects as go

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_sizes, y=train_scores_mean,
                    mode='lines+markers',
                    name='Training score'))
fig.add_trace(go.Scatter(x=train_sizes, y=test_scores_mean,
                    mode='lines+markers',
                    name='Cross Validation score'))

fig.show()

In [ ]:
test_pred_LR=tuned_LR.predict(poly_transformed_test)

#test_pred_SVM=tuned_SVM.predict(poly_transformed_test)

test_pred_Cat=tuned_cat.predict(poly_transformed_test)

test_pred_RF=tuned_RF.predict(poly_transformed_test)

test_pred_df=pd.DataFrame(test_pred_LR,columns=["LR"])
test_pred_df['Cat']=test_pred_Cat
test_pred_df['RF']=test_pred_RF

test_pred_df['final']=test_pred_df.apply(lambda x: x.LR*x.Cat*x.RF,axis=1)

#predictions=tuned_XGB.predict(test_pred_df)

CatBoostError: ignored

In [ ]:
from sklearn.ensemble import VotingClassifier

vc=VotingClassifier([('RF',tuned_RF),('LR',tuned_LR)],voting='soft')
vc.fit(poly_transformed_train,y_train)
pred=vc.predict(poly_transformed_test)

In [ ]:
res= pd.DataFrame(pred)

res.index=test_df.index
res.columns= ['prediction']

res.to_csv('Tuned RF+LR.csv',index=False)

## PyCaret

In [ ]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
"""
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
"""

'\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import f1_score\nfrom sklearn.metrics import classification_report\nfrom sklearn.preprocessing import RobustScaler\nfrom sklearn.feature_selection import SelectFromModel\nfrom sklearn.svm import SVC\nfrom sklearn.preprocessing import PowerTransformer\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.metrics import classification_report\nfrom sklearn.preprocessing import PolynomialFeatures\nfrom sklearn.preprocessing import MinMaxScaler\nfrom sklearn.preprocessing import QuantileTransformer\n'

In [ ]:
train_df  = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/Heart_Disease/Training_set_heart.csv" )

train_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,48,1,2,124,255,1,1,175,0,0.0,2,2,2,1
1,68,0,2,120,211,0,0,115,0,1.5,1,0,2,1
2,46,1,0,120,249,0,0,144,0,0.8,2,0,3,0
3,60,1,0,130,253,0,1,144,1,1.4,2,1,3,0
4,43,1,0,115,303,0,1,181,0,1.2,1,0,2,1


In [ ]:
test_df = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/Heart_Disease/Testing_set_heart.csv')

In [ ]:
!pip install pycaret

     |████████████████████████████████| 256kB 5.5MB/s 
     |████████████████████████████████| 157.5MB 87kB/s 
     |████████████████████████████████| 1.8MB 41.2MB/s 
     |████████████████████████████████| 13.9MB 38.5MB/s 
     |████████████████████████████████| 102kB 11.0MB/s 
     |████████████████████████████████| 6.8MB 41.5MB/s 
     |████████████████████████████████| 266kB 33.6MB/s 
     |████████████████████████████████| 1.6MB 44.9MB/s 
     |████████████████████████████████| 276kB 39.4MB/s 
     |████████████████████████████████| 174kB 46.8MB/s 
     |████████████████████████████████| 66.3MB 89kB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 163kB 45.1MB/s 
     |████████████████████████████████| 1.1MB 42.8MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 153kB 50.3MB/s 
     |████████████████████████████████| 337kB 45.3MB/s 
     |████████████████████████████████| 2.1MB 40.9MB/

In [ ]:
from pycaret.classification import *

In [ ]:
train_df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [ ]:
train_df.columns[:-1].to_list()

['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal']

In [ ]:
"""
Below code gives F1 score of 88.46153846153845 on test set
"""
s= setup(train_df,target='target',normalize=True,transformation=True,normalize_method='robust' ,
         polynomial_features=True, remove_multicollinearity=True,transformation_method='quantile',
      trigonometry_features=True,feature_selection=True,polynomial_degree=2,session_id=1,
      numeric_features=train_df.columns[:-1].to_list(), feature_interaction=True)

rf=create_model('rf')
cat=create_model('catboost')
select_models =[rf,cat]

tuned_models=[tune_model(i,optimize='F1') for i in select_models]  

blender = blend_models(tuned_models,optimize='F1',method='hard')

predictions=predict_model(blender,data=test_df)['Label']

res=pd.DataFrame(predictions)

res.index=test_df.index

res.columns=['prediction']

res.to_csv('PyCaret_blender_robust_quant_degree_2.csv',index=False)

## Final Model - F1 score of 89.90825688073394

In [ ]:
s= setup(train_df,target='target',normalize=True,transformation=True,normalize_method='robust' ,
         polynomial_features=True, remove_multicollinearity=True,transformation_method='quantile',
      trigonometry_features=True,feature_selection=True,polynomial_degree=2,session_id=1,
      numeric_features=train_df.columns[:-1].to_list(), feature_interaction=True)

rf=create_model('rf')
cat=create_model('catboost')

tuned_rf=tune_model(rf,optimize='F1') 
tuned_cat=tune_model(cat,optimize='F1') 

RF_pred=predict_model(tuned_rf,data=train_df)['Label']
Cat_pred=predict_model(tuned_cat,data=train_df)['Label']

NN_input=pd.DataFrame(RF_pred)
NN_input.columns=['RF']
NN_input['Cat']=Cat_pred

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8000,0.9464,0.7500,0.8571,0.8000,0.6018,0.6071
1,0.9333,0.9643,0.8750,1.0000,0.9333,0.8673,0.8750
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.8667,0.8750,0.8750,0.8750,0.8750,0.7321,0.7321
4,0.8667,0.9286,1.0000,0.8000,0.8889,0.7273,0.7559
5,0.7333,0.7321,0.8750,0.7000,0.7778,0.4545,0.4725
6,0.8000,0.8571,0.8750,0.7778,0.8235,0.5946,0.6001
7,0.9333,1.0000,0.8750,1.0000,0.9333,0.8673,0.8750
8,0.9286,0.9184,1.0000,0.8750,0.9333,0.8571,0.8660
9,0.9286,1.0000,1.0000,0.8750,0.9333,0.8571,0.8660


In [ ]:
NN_input.shape

(212, 3)

In [ ]:
classifier_ann = Sequential()
classifier_ann.add(Dense(100,activation='relu',kernel_initializer='he_uniform',input_dim=NN_input.shape[1]))
classifier_ann.add(Dropout(0.5,seed=123))
classifier_ann.add(BatchNormalization())
classifier_ann.add(Dense(100,activation='relu',kernel_initializer='he_uniform'))
classifier_ann.add(Dropout(0.5,seed=123))
classifier_ann.add(BatchNormalization())
classifier_ann.add(Dense(1,activation='sigmoid',kernel_initializer='glorot_uniform'))
classifier_ann.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_103 (Dense)            (None, 100)               400       
_________________________________________________________________
dropout_78 (Dropout)         (None, 100)               0         
_________________________________________________________________
batch_normalization_77 (Batc (None, 100)               400       
_________________________________________________________________
dense_104 (Dense)            (None, 100)               10100     
_________________________________________________________________
dropout_79 (Dropout)         (None, 100)               0         
_________________________________________________________________
batch_normalization_78 (Batc (None, 100)               400       
_________________________________________________________________
dense_105 (Dense)            (None, 1)               

In [ ]:
classifier_ann.compile(optimizer='adam',metrics=keras.metrics.AUC(name='auc'),loss='binary_crossentropy')

#early stopping to avoid overfitting

es=EarlyStopping(monitor='val_auc',patience=10,mode='max')

model_history=classifier_ann.fit(NN_input,y,epochs=100,
                   validation_split=0.2,
                   callbacks=es)

Epoch 1/100
6/6 [==============================] - 1s 74ms/step - loss: 0.7385 - auc: 0.6486 - val_loss: 0.3716 - val_auc: 1.0000
Epoch 2/100
6/6 [==============================] - 0s 12ms/step - loss: 0.6043 - auc: 0.7834 - val_loss: 0.3241 - val_auc: 1.0000
Epoch 3/100
6/6 [==============================] - 0s 12ms/step - loss: 0.4183 - auc: 0.8999 - val_loss: 0.2802 - val_auc: 1.0000
Epoch 4/100
6/6 [==============================] - 0s 12ms/step - loss: 0.3651 - auc: 0.9137 - val_loss: 0.2443 - val_auc: 1.0000
Epoch 5/100
6/6 [==============================] - 0s 12ms/step - loss: 0.2407 - auc: 0.9571 - val_loss: 0.2134 - val_auc: 1.0000
Epoch 6/100
6/6 [==============================] - 0s 12ms/step - loss: 0.3230 - auc: 0.9276 - val_loss: 0.1900 - val_auc: 1.0000
Epoch 7/100
6/6 [==============================] - 0s 13ms/step - loss: 0.3350 - auc: 0.9112 - val_loss: 0.1704 - val_auc: 1.0000
Epoch 8/100
6/6 [==============================] - 0s 12ms/step - loss: 0.2656 - auc: 0.94

In [ ]:
fig=go.Figure()

x=[i for i in range(1,12)]

fig.add_trace(go.Scatter(x=x,y=model_history.history['auc'],mode='lines',name='Training AUC'))
fig.add_trace(go.Scatter(x=x,y=model_history.history['val_auc'],mode='lines',name='Valid AUC'))
fig.add_trace(go.Scatter(x=x,y=model_history.history['loss'],mode='lines',name='Training loss'))
fig.add_trace(go.Scatter(x=x,y=model_history.history['val_loss'],mode='lines',name='Valid loss'))

fig.show()

In [ ]:
RF_pred=predict_model(tuned_rf,data=test_df)['Label']
Cat_pred=predict_model(tuned_cat,data=test_df)['Label']

NN_input_test=pd.DataFrame(RF_pred)
NN_input_test.columns=['RF']
NN_input_test['Cat']=Cat_pred

test_predictions=classifier_ann.predict_classes(NN_input_test)

res=pd.DataFrame(test_predictions)

res.index=test_df.index

res.columns=['prediction']

res.to_csv('PyCaret+ANN.csv',index=False)

In [ ]:
s= setup(train_df,target='target',normalize=True,transformation=True,normalize_method='robust' ,
         polynomial_features=True, remove_multicollinearity=True,transformation_method='quantile',
      trigonometry_features=True,feature_selection=True,polynomial_degree=2,session_id=1,
      numeric_features=train_df.columns[:-1].to_list(), feature_interaction=True)

,Description,Value
0,session_id,1
1,Target,target
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(212, 14)"
5,Missing Values,False
6,Numeric Features,13
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
top2=compare_models(sort='F1',n_select = 2)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8786,0.9219,0.9000,0.8792,0.8866,0.7564,0.7627,1.795
ada,Ada Boost Classifier,0.8657,0.9230,0.8732,0.8835,0.8749,0.7296,0.7354,0.099
ridge,Ridge Classifier,0.8648,0.0000,0.8839,0.8731,0.8741,0.7278,0.7360,0.016
lda,Linear Discriminant Analysis,0.8581,0.9242,0.8839,0.8633,0.8690,0.7141,0.7228,0.020
lightgbm,Light Gradient Boosting Machine,0.8524,0.9148,0.8875,0.8549,0.8660,0.7018,0.7107,0.168
knn,K Neighbors Classifier,0.8581,0.8963,0.8607,0.8764,0.8658,0.7155,0.7209,0.119
gbc,Gradient Boosting Classifier,0.8519,0.9046,0.8750,0.8607,0.8636,0.7021,0.7103,0.101
rf,Random Forest Classifier,0.8514,0.9172,0.8732,0.8556,0.8622,0.7013,0.7054,0.465
nb,Naive Bayes,0.8514,0.9125,0.8589,0.8597,0.8577,0.7018,0.7048,0.017
xgboost,Extreme Gradient Boosting,0.8452,0.9153,0.8607,0.8536,0.8532,0.6892,0.6969,0.175


In [ ]:
#select_models =[lda,rf,lr,gbc]

tuned_models=[tune_model(i,optimize='F1') for i in top2]  

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9333,0.9643,0.8750,1.0000,0.9333,0.8673,0.8750
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.8667,0.8929,0.8750,0.8750,0.8750,0.7321,0.7321
4,0.8667,0.8571,1.0000,0.8000,0.8889,0.7273,0.7559
5,0.7333,0.8393,0.7500,0.7500,0.7500,0.4643,0.4643
6,0.8000,0.8571,0.8750,0.7778,0.8235,0.5946,0.6001
7,0.9333,0.9643,0.8750,1.0000,0.9333,0.8673,0.8750
8,0.8571,0.9592,0.8571,0.8571,0.8571,0.7143,0.7143
9,0.9286,0.9592,1.0000,0.8750,0.9333,0.8571,0.8660


In [ ]:
lgbm=create_model('lightgbm')
lda=create_model('lda')
select_models =[lgbm,lda]

tree_tuned_models=[tune_model(i,optimize='F1') for i in select_models]  

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.8667,0.9286,0.8750,0.8750,0.8750,0.7321,0.7321
2,0.8000,0.9821,0.6250,1.0000,0.7692,0.6087,0.6614
3,0.8667,0.8571,0.8750,0.8750,0.8750,0.7321,0.7321
4,0.8667,0.9286,1.0000,0.8000,0.8889,0.7273,0.7559
5,0.6667,0.7679,0.7500,0.6667,0.7059,0.3243,0.3273
6,0.8667,0.9643,0.8750,0.8750,0.8750,0.7321,0.7321
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
9,0.7857,0.9388,0.8571,0.7500,0.8000,0.5714,0.5774


In [ ]:
tuned_models+tree_tuned_models

 AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=0.406,
                    n_estimators=230, random_state=1),
 LGBMClassifier(bagging_fraction=0.9, bagging_freq=1, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=0.6,
                importance_type='split', learning_rate=0.205, max_depth=-1,
                min_child_samples=15, min_child_weight=0.001, min_split_gain=0.2,
                n_estimators=240, n_jobs=-1, num_leaves=80, objective=None,
                random_state=1, reg_alpha=0.2, reg_lambda=1, silent=True,
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=0.01,
                            solver='lsqr', store_covariance=False, tol=0.0001)]

In [ ]:
blender = blend_models(tuned_models+tree_tuned_models,optimize='F1',method='hard')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9333,0.0,0.8750,1.0000,0.9333,0.8673,0.8750
1,0.9333,0.0,0.8750,1.0000,0.9333,0.8673,0.8750
2,1.0000,0.0,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.8667,0.0,0.8750,0.8750,0.8750,0.7321,0.7321
4,0.8667,0.0,1.0000,0.8000,0.8889,0.7273,0.7559
5,0.6667,0.0,0.7500,0.6667,0.7059,0.3243,0.3273
6,0.8667,0.0,0.8750,0.8750,0.8750,0.7321,0.7321
7,0.9333,0.0,0.8750,1.0000,0.9333,0.8673,0.8750
8,0.9286,0.0,1.0000,0.8750,0.9333,0.8571,0.8660
9,0.8571,0.0,0.8571,0.8571,0.8571,0.7143,0.7143


In [ ]:
stacker = stack_models(tuned_models+tree_tuned_models,optimize='F1',meta_model=lda )                 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8000,0.9286,0.7500,0.8571,0.8000,0.6018,0.6071
1,0.8667,0.8750,0.8750,0.8750,0.8750,0.7321,0.7321
2,0.9333,1.0000,0.8750,1.0000,0.9333,0.8673,0.8750
3,0.7333,0.8036,0.7500,0.7500,0.7500,0.4643,0.4643
4,0.8667,0.8571,1.0000,0.8000,0.8889,0.7273,0.7559
5,0.7333,0.8036,0.8750,0.7000,0.7778,0.4545,0.4725
6,0.8000,0.8393,0.8750,0.7778,0.8235,0.5946,0.6001
7,0.8667,0.9643,0.8750,0.8750,0.8750,0.7321,0.7321
8,0.9286,0.9184,1.0000,0.8750,0.9333,0.8571,0.8660
9,0.7857,0.9184,0.8571,0.7500,0.8000,0.5714,0.5774


In [ ]:
best_f1_model = automl(optimize = 'F1')

In [ ]:
best_f1_model

AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=0.406,
                   n_estimators=230, random_state=1)

In [ ]:
#evaluate_model(best_f1_model)  

In [ ]:
#lda=create_model('lda')

In [ ]:
#tuned_model=tune_model(lda,optimize='F1')

In [ ]:
#evaluate_model(tuned_model)

In [ ]:
#ensembled_lda=ensemble_model(tuned_model,optimize='F1')

In [ ]:
#evaluate_model(ensembled_lda)

In [ ]:
#lr=create_model('lr')

In [ ]:
#tuned_lr=tune_model(lr,optimize='F1')

In [ ]:
#evaluate_model(tuned_lr)

In [ ]:
#ensembled_lr=ensemble_model(tuned_lr,optimize='F1')

In [ ]:
#blender=blend_models([ensembled_lda,ensembled_lr],optimize='F1')

In [ ]:
predictions=predict_model(blender,data=test_df)['Label']

res=pd.DataFrame(predictions)

res.index=test_df.index

res.columns=['prediction']

res.to_csv('PyCaret_blender_top3_robust_quant_degree_2.csv',index=False)

In [ ]:
predictions=predict_model(best_f1_model,data=test_df)['Label']

res=pd.DataFrame(predictions)

res.index=test_df.index

res.columns=['prediction']

res.to_csv('PyCaret_best_f1_model.csv',index=False)